<a href="https://colab.research.google.com/github/DenisAraujo68/finetune_Galois/blob/master/Finetuning_of_a_Galois_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Finetune a galois Model w/ GPU For Free 

by [Denis Araujo](https://www.linkedin.com/in/denisandreiaraujo/)

*Last updated: Mar 30th, 2020*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
!ls -al models

total 16
drwxr-xr-x 4 root root 4096 Mar 30 20:36 .
drwxr-xr-x 1 root root 4096 Mar 30 20:37 ..
drwxr-xr-x 2 root root 4096 Mar 30 20:36 117M
drwxr-xr-x 2 root root 4096 Mar 30 20:36 345M


In [0]:
!pip uninstall -qy tensorflow
!pip uninstall -qy tensorflow_estimator
!pip install -q tensorflow==1.14.0
!pip install -q tensorflow_estimator
!pip install -q gpt-2-simple
#!cp /content/gpt_2.py /usr/local/lib/python3.6/dist-packages/gpt_2_simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

     |████████████████████████████████| 109.2MB 96kB/s 
     |████████████████████████████████| 491kB 54.6MB/s 
     |████████████████████████████████| 3.2MB 73.3MB/s 


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

## GPU

Colaboratory now uses an Nvidia T4 GPU, which is slightly faster than the old Nvidia GPUs for training Galois on GPT-2, and has more memory allowing you to train the larger GPT-2 models.

You can verify which GPU is active by running the cell below.

In [0]:
!nvidia-smi

Mon Mar 30 20:30:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are two sizes of GPT-2:

* `117M` (default): the "small" model, 500MB on disk.
* `345M`: the "medium" model, 1.5GB on disk.

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
gpt2.download_gpt2(model_name="345M")
gpt2.download_gpt2(model_name="117M")

Fetching checkpoint: 1.05Mit [00:00, 240Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 116Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 676Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:10, 133Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 544Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 89.4Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 169Mit/s]                                                       
Fetching checkpoint: 1.05Mit [00:00, 281Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 109Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 397Mit/s]                                           

## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [0]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [0]:
# file_name = "/content/drive/My Drive/text_encoded7_1.npz"
file_name = "/content/drive/My Drive/CWI/PeI/Galois/shakespeare.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [0]:
# file_name = "/content/drive/My Drive/files"
# gpt2.encode_dataset(file_name)
# file_name = "text_encoded.npz"
# !ls -lh
# !cat /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py
# mv ./checkpoint/run8 ./checkpoint/run9

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [0]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='345M',
              steps=-1,
              learning_rate=0.00001,
#               sgd=True,
#               accumulate_gradients=10,
              use_memory_saving_gradients=True,
              restore_from='fresh',
              run_name='run10',
              print_every=10,
              sample_every=200,
#               overwrite=True,
              save_every=300
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/345M/model.ckpt
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.90s/it]


dataset has 338025 tokens
Training...
[10 | 808.00] loss=3.26 avg=3.26
[20 | 1609.16] loss=3.55 avg=3.41
[30 | 2353.17] loss=2.87 avg=3.22
[40 | 3075.41] loss=3.13 avg=3.20
[50 | 3779.68] loss=3.22 avg=3.20
[60 | 4490.00] loss=3.44 avg=3.25
[70 | 5211.08] loss=3.41 avg=3.27
[80 | 5924.68] loss=3.39 avg=3.29
[90 | 6635.21] loss=2.94 avg=3.25
[100 | 7337.65] loss=3.85 avg=3.31
[110 | 8037.94] loss=3.17 avg=3.30
[120 | 8743.97] loss=3.11 avg=3.28
[130 | 9448.91] loss=3.24 avg=3.28
[140 | 10158.73] loss=3.34 avg=3.28
[150 | 10864.65] loss=3.16 avg=3.27
[160 | 11569.14] loss=3.33 avg=3.28


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name='run9')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
gpt2.copy_checkpoint_from_gdrive(run_name='run9')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
# import gpt_2_simple as gpt2
# from datetime import datetime
# from google.colab import files
tf.reset_default_graph()
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run9')

W0725 21:51:40.111256 140168434247552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/src/memory_saving_gradients.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0725 21:51:40.118145 140168434247552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py:90: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0725 21:51:40.119478 140168434247552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py:100: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0725 21:51:40.350766 140168434247552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py:340: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 21:51:40.354785 140168434247552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-pac

Loading checkpoint checkpoint/run9/model-19500


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [0]:
gpt2.generate(sess, length=1023, temperature=0.8, run_name='run1')

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [0]:
gpt2.generate(sess,
              length=5,
              top_p=.9,
              temperature=.8,
              seed=99,
              prefix="""import os
import sys
# Count lines of codes in the given directory, separated by file extension.
def main(directory):
  line_count = {}
  for filename in os.listdir(directory):
    _, ext = os.path.splitext(filename)
    if ext not""",
              nsamples=5,
              batch_size=5,
              run_name='run9'
              )

import os
import sys
# Count lines of codes in the given directory, separated by file extension.
def main(directory):
  line_count = {}
  for filename in os.listdir(directory):
    _, ext = os.path.splitext(filename)
    if ext not in os.path:
import os
import sys
# Count lines of codes in the given directory, separated by file extension.
def main(directory):
  line_count = {}
  for filename in os.listdir(directory):
    _, ext = os.path.splitext(filename)
    if ext not in sys.path:
import os
import sys
# Count lines of codes in the given directory, separated by file extension.
def main(directory):
  line_count = {}
  for filename in os.listdir(directory):
    _, ext = os.path.splitext(filename)
    if ext not in os.path.
import os
import sys
# Count lines of codes in the given directory, separated by file extension.
def main(directory):
  line_count = {}
  for filename in os.listdir(directory):
    _, ext = os.path.splitext(filename)
    if ext not in sys.extpath
import os
import sys

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp and then download it.

You can rerun the cell as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

files.download(gen_file)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail or out-of-memory/OOM), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.